# 损失函数类

损失函数是梯度计算的起点。

In [1]:
from abc import abstractmethod, ABC
import numpy as np

## 基础架构

### 张量

In [2]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)

    def size(self):
        return len(self.data)

    def __str__(self):
        return str(self.data)

### 基础层

In [3]:
class Layer(ABC):

    def __call__(self, x: Tensor):
        return self.forward(x)

    @abstractmethod
    def forward(self, x: Tensor):
        pass

### 基础损失函数

和基础层一样，基础损失函数也是一个抽象类，定义了一个计算梯度的接口。

In [4]:
class Loss(ABC):

    def __call__(self, p: Tensor, y: Tensor):
        return self.loss(p, y)

    @abstractmethod
    def loss(self, p: Tensor, y: Tensor):
        pass

## 数据

### 特征、标签

In [5]:
feature = Tensor([28.1, 58.0])
label = Tensor([165])

## 模型

### 线性层

In [6]:
class Linear(Layer):

    def __init__(self, in_size, out_size):
        self.weight = Tensor(np.ones((out_size, in_size)) / in_size)
        self.bias = Tensor(np.zeros(out_size))

    def forward(self, x: Tensor):
        return Tensor(x.data @ self.weight.data.T + self.bias.data)

### 损失函数（平均平方差）

损失函数（平均平方差）继承了基础损失函数的虚拟接口，封装了计算平均平方差的功能。这是回归模型最常用的损失函数。

In [7]:
class MSELoss(Loss):

    def loss(self, p: Tensor, y: Tensor):
        return Tensor(np.mean(np.square(y.data - p.data)))

## 验证

### 推理

In [8]:
layer = Linear(feature.size(), 1)
prediction = layer(feature)
print(f'prediction: {prediction}')

prediction: [43.05]


### 评估

In [9]:
loss = MSELoss()
error = loss(prediction, label)
print(f'error: {error}')

error: 14871.802500000002
